In [1]:
import opendatasets as od
import zipfile
import requests
import pandas as pd
import seaborn as sns
from word2number import w2n
import regex as re
import numpy as np

In [2]:
meta_df = pd.read_csv('the-movies-dataset/movies_metadata.csv', low_memory=False)
budget_df = pd.read_csv('budgets.csv', index_col=0)
revenue_df = pd.read_csv('revenues.csv', index_col=0)

In [36]:
first_df = meta_df[(meta_df['revenue'].astype(float) != 0) & (meta_df['budget'].astype(float) != 0)]

ValueError: could not convert string to float: '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'

In [4]:
meta_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
budget_df.dropna(inplace=True)
budget_df.shape

(5533, 2)

In [6]:
revenue_df.shape

(3403, 2)

In [7]:
first_df.shape

(7414, 24)

In [8]:
first_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0


In [9]:
budget_df.budget_str.value_counts()

$5 million                                                              113
$10 million                                                             105
$3 million                                                               92
$2 million                                                               91
$1 million                                                               86
                                                                       ... 
$2,467,000                                                                1
€4.4 million                                                              1
$788,000                                                                  1
[[Deutsche Mark|DEM]] 4,300,000<br /> (approx. [[Euro|€]] 2,200,000)      1
{{INR|29 [[crore]]|link|=|yes}}                                           1
Name: budget_str, Length: 2747, dtype: int64

In [10]:
float(revenue_df.revenue_str[0].lstrip('$').rstrip(' million')) * w2n.word_to_num(revenue_df.revenue_str[0])

71500000.0

In [11]:
revenue_df['revenue'] = np.nan
for ii in range(revenue_df.shape[0]):
    try:
        revenue_df['revenue'][ii] = float(revenue_df.revenue_str[ii].lstrip('$').rstrip(' million')) * w2n.word_to_num(revenue_df.revenue_str[ii])
    except:
        try:
            revenue_df['revenue'][ii] = float(revenue_df.revenue_str[ii].lstrip('$').replace(',' , ''))
        except:
            pass
        
revenue_df

C:\Users\Jarrod\AppData\Local\Temp\ipykernel_10064\1938443925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenue_df['revenue'][ii] = float(revenue_df.revenue_str[ii].lstrip('$').rstrip(' million')) * w2n.word_to_num(revenue_df.revenue_str[ii])
C:\Users\Jarrod\AppData\Local\Temp\ipykernel_10064\1938443925.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenue_df['revenue'][ii] = float(revenue_df.revenue_str[ii].lstrip('$').replace(',' , ''))


,title,revenue_str,revenue
0,Grumpier Old Men,$71.5 million,71500000.0
1,Dracula: Dead and Loving It,$10.7 million,10700000.0
2,Copycat,$79 million,79000000.0
3,Powder,$30.9 million,30900000.0
4,Wings of Courage,"$15,054,636",15054636.0
...,...,...,...
3398,Frankenstein Created Woman,"457,019 admissions (France)",NaN
3399,Up 'n' Under,US$12 million,NaN
3400,The Pope Must Die,"£1,150,000 (UK) <br>$582,510 (US)",NaN
3401,Phillauri,{{estimation}} {{INR|46.6 [[crore]]|link|=|yes}},NaN


In [12]:
revenue_df['revenue'].isna().sum()

1269

In [13]:
revenue_df.dropna(inplace=True)
revenue_df.shape

(2134, 3)

In [14]:
budget_df['budget'] = np.nan
for ii in range(budget_df.shape[0]):
    try:
        budget_df['budget'][ii] = float(budget_df.budget_str[ii].lstrip('$').rstrip(' million')) * w2n.word_to_num(budget_df.budget_str[ii])
    except:
        try:
            budget_df['budget'][ii] = float(budget_df.budget_str[ii].lstrip('$').replace(',' , ''))
        except:
            pass
        
budget_df

C:\Users\Jarrod\AppData\Local\Temp\ipykernel_10064\2169771460.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget_df['budget'][ii] = float(budget_df.budget_str[ii].lstrip('$').rstrip(' million')) * w2n.word_to_num(budget_df.budget_str[ii])
C:\Users\Jarrod\AppData\Local\Temp\ipykernel_10064\2169771460.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget_df['budget'][ii] = float(budget_df.budget_str[ii].lstrip('$').replace(',' , ''))


,title,budget_str,budget
0,Grumpier Old Men,$25 million,25000000.0
1,Father of the Bride Part II,$30 million {{Citation needed|date|=|September...,NaN
2,Dracula: Dead and Loving It,$30 million,30000000.0
3,Balto,$31 million,31000000.0
4,Copycat,$20 million,20000000.0
...,...,...,...
5531,The Pope Must Die,£2.5 million,NaN
5532,Carry On England,"£250,000",NaN
5533,Phillauri,{{INR|29 [[crore]]|link|=|yes}},NaN
5534,The Final Storm,$5 million,NaN


In [15]:
budget_df.isna().sum()

title            0
budget_str       0
budget        2040
dtype: int64

In [16]:
budget_df.dropna(inplace=True)

In [17]:
df = pd.merge(revenue_df, budget_df, on='title')
df

,title,revenue_str,revenue,budget_str,budget
0,Grumpier Old Men,$71.5 million,71500000.0,$25 million,25000000.0
1,Dracula: Dead and Loving It,$10.7 million,10700000.0,$30 million,30000000.0
2,Copycat,$79 million,79000000.0,$20 million,20000000.0
3,Powder,$30.9 million,30900000.0,$9.5 million,9500000.0
4,Wings of Courage,"$15,054,636",15054636.0,$20 million,20000000.0
...,...,...,...,...,...
1825,Wish Upon,$23.5 million,23500000.0,$12 million,12000000.0
1826,Girls Trip,$140.9 million,140900000.0,$19 million,19000000.0
1827,Landline,"$940,854",940854.0,$3.7 million,3700000.0
1828,Julia Misbehaves,"$4,497,000",4497000.0,"$2,706,000",2706000.0


In [18]:
budget_df.dtypes

title          object
budget_str     object
budget        float64
dtype: object

In [19]:
revenue_df.dtypes

title           object
revenue_str     object
revenue        float64
dtype: object

In [20]:
df[df.title=='Grumpier Old Men']

,title,revenue_str,revenue,budget_str,budget
0,Grumpier Old Men,$71.5 million,71500000.0,$25 million,25000000.0


In [21]:
df.drop(['revenue_str', 'budget_str'], axis=1, inplace=True)

In [22]:
df

,title,revenue,budget
0,Grumpier Old Men,71500000.0,25000000.0
1,Dracula: Dead and Loving It,10700000.0,30000000.0
2,Copycat,79000000.0,20000000.0
3,Powder,30900000.0,9500000.0
4,Wings of Courage,15054636.0,20000000.0
...,...,...,...
1825,Wish Upon,23500000.0,12000000.0
1826,Girls Trip,140900000.0,19000000.0
1827,Landline,940854.0,3700000.0
1828,Julia Misbehaves,4497000.0,2706000.0


In [23]:
pd.set_option('display.max_columns', None)
second_df = meta_df[meta_df.title.isin(df.title)].copy()
second_df.drop(['budget', 'revenue'], inplace=True, axis=1)
second_df

,adult,belongs_to_collection,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
11,False,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,5.430331,/xve4cgfYItnOhtzLYoTwTVy5FGr.jpg,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1995-12-22,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0
16,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,4584,tt0114388,en,Sense and Sensibility,"Rich Mr. Dashwood dies, leaving his second wif...",10.673167,/lA9HTy84Bb6ZwNeyoZKobcMdpMc.jpg,"[{'name': 'Columbia Pictures Corporation', 'id...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-12-13,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Lose your heart and come to your senses.,Sense and Sensibility,False,7.2,364.0
21,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,1710,tt0112722,en,Copycat,An agoraphobic psychologist and a female detec...,10.701801,/80czeJGSoik22fhtUM9WzyjUU4r.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-27,124.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One man is copying the most notorious killers ...,Copycat,False,6.5,199.0
23,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",NaN,12665,tt0114168,en,Powder,Harassed by classmates who won't accept his sh...,12.133094,/1uRKsxOCtgz0xVqs9l4hYtp4dFm.jpg,"[{'name': 'Caravan Pictures', 'id': 175}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-27,111.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An extraordinary encounter with another human ...,Powder,False,6.3,143.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45202,False,NaN,"[{'id': 14, 'name': 'Fantasy'}, {'id': 27, 'na...",http://wishuponmovie.com/,440597,tt5322012,en,Wish Upon,A teenage girl discovers a box with magical po...,59.578823,/u0vnocj57vJt5DHoBEqUOD1G4SU.jpg,"[{'name': 'Broad Green Pictures', 'id': 49389}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2017-07-07,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Be careful what you wish for,Wish Upon,False,5.3,127.0
45204,False,NaN,"[{'id': 35, 'name': 'Comedy'}]",NaN,417870,tt3564472,en,Girls Trip,Four girlfriends take a trip to New Orleans fo...,37.964872,/94nCLn74lBh4pLdZPIad18BpheE.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-07-21,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"""Forgive us in advance for this wild weekend""",Girls Trip,False,7.1,393.0
45205,False,NaN,"[{'id': 35, 'name': 'Comedy'}]",NaN,419459,tt5737862,en,Landline,A teenager living with her sister and parents ...,1.811506,/p7RVsyi67S95RyNJjX99E1MaOsA.jpg,"[{'name': 'Magnolia Pictures', 'id': 1030}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-07-21,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,1995. When people were harder to reach.,Landline,False,5.1,9.0
45237,False,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,69483,tt0040498,en,Julia Misbehaves,Julia and Will

In [24]:
second_df = second_df.merge(df, how='left', on='title')
second_df.head()

,adult,belongs_to_collection,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,revenue,budget
0,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,71500000.0,25000000.0
1,False,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,5.430331,/xve4cgfYItnOhtzLYoTwTVy5FGr.jpg,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1995-12-22,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0,10700000.0,30000000.0
2,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,4584,tt0114388,en,Sense and Sensibility,"Rich Mr. Dashwood dies, leaving his second wif...",10.673167,/lA9HTy84Bb6ZwNeyoZKobcMdpMc.jpg,"[{'name': 'Columbia Pictures Corporation', 'id...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-12-13,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Lose your heart and come to your senses.,Sense and Sensibility,False,7.2,364.0,135000000.0,16000000.0
3,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,1710,tt0112722,en,Copycat,An agoraphobic psychologist and a female detec...,10.701801,/80czeJGSoik22fhtUM9WzyjUU4r.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-27,124.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One man is copying the most notorious killers ...,Copycat,False,6.5,199.0,79000000.0,20000000.0
4,False,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",NaN,12665,tt0114168,en,Powder,Harassed by classmates who won't accept his sh...,12.133094,/1uRKsxOCtgz0xVqs9l4hYtp4dFm.jpg,"[{'name': 'Caravan Pictures', 'id': 175}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-27,111.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An extraordinary encounter with another human ...,Powder,False,6.3,143.0,30900000.0,9500000.0


In [25]:
final_df = pd.concat([first_df, second_df])

In [26]:
final_df.shape

(9345, 24)

In [27]:
final_df.to_csv('webscraped dataset')

In [31]:
df[df.title == 'Dangerous Minds']

,title,revenue,budget


ValueError: could not convert string to float: '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'